**Ground up 3D neural network for ADNI alzheimers classification**

In [1]:
import math
import json

import cv2

import numpy as np
import numpy.ma as ma

import scipy as scipy

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap

from mpl_toolkits.axes_grid1 import make_axes_locatable

import itertools

import tensorflow as tf

import keras
from keras import backend as K
from keras.utils import np_utils
from keras.engine import Layer

from keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling2D, Deconvolution2D, UpSampling2D, Reshape, Flatten, ZeroPadding2D, BatchNormalization, Lambda, Dropout, Activation
from keras.layers import Conv3D, MaxPooling3D
from keras.models import Model, Sequential
from keras.models import model_from_json

from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing import image

from keras.callbacks import Callback
from keras.models import load_model

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

#from vis.utils import utils
#from vis.visualization import visualize_saliency

from mpl_toolkits.mplot3d import Axes3D

import nibabel as nib
from keras.utils import to_categorical

from scipy.ndimage import zoom

import os

from sklearn.model_selection import train_test_split

from sklearn import svm, datasets
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

import random

from sklearn.utils import class_weight

import theano
import theano.tensor as T

from keras.regularizers import l2

theano.config.opennp = True


Using TensorFlow backend.


In [2]:
#Parameters (Modify as needed)
img_size_x = 96
img_size_y = 96
img_size_z = 62

batch_size = 8
nb_classes = 3
nb_epoch = 50

c = 0

early_stopping_patience = 20

class_names = ["CN", "MCI", "AD"]

In [3]:
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")

In [4]:
#Dont use this one
def load_dataset(directory):
    
    patientData = np.loadtxt("ADNI1_Complete_1Yr_1.5T_10_26_2019.csv", 
                         dtype= 'str', skiprows=1, delimiter=',')
    
    integer_mapping = {x: i for i,x in enumerate(['AD', 'MCI', 'CN'])}
    y = np.asarray([integer_mapping[word] for word in patientData[:,1]])
    labels = to_categorical(y, num_classes=3, dtype='float32')
    
    
    
    xdim = 96
    ydim = 96
    zdim = 62
    X = np.zeros((306,xdim,ydim,zdim,1))
    Y = np.zeros(306)
    
    l = os.listdir(directory)
    random.shuffle(l)
    
    countHC = 0
    countMCI = 0
    countAD = 0
    
    objindex = 0
    for i, filename in enumerate(l):
        if (filename.startswith('.')):
            print ("hidden file")
        else:
            subject = filename[:-4]
            
            ind = np.where(patientData[:,0] == subject[:-1])[0][0]
            y = labels[ind, :]
            y = y.reshape(1,-1)

            if str(y) == "[[1. 0. 0.]]":
                y = 1
            if str(y) == "[[0. 1. 0.]]":
                continue
            if str(y) == "[[0. 0. 1.]]":
                y = 0
            
            Y[objindex] = y
            objindex += 1

            epi_img = nib.load(directory + subject + '.nii')
            # Get voxel array
            epi_img_data = epi_img.get_fdata()
            n_i, n_j, n_k = epi_img_data.shape

            #if (n_i != 192 or n_j != 192):
                #epi_img_data = cv2.resize(epi_img_data, (192, 192))

            '''
            if (n_i != 192 or n_j != 192 or n_k != 160):
                epi_img_data = cv2.resize(epi_img_data, (192, 192))
                #epi_img_data = epi_img_data[0:192,0:192,int(n_k/2)-80:int(n_k/2)+80]

                #resampling to make all MRI volumes the same dimensions
                epi_img_data = zoom(epi_img_data, (float(xdim/n_i), float(ydim/n_j), float(zdim/n_k)), order = 0)'''

            x = zoom(epi_img_data, (float(xdim/n_i), float(ydim/n_j), float(zdim/n_k)), order = 1)
            
            x = (x-x.min())/x.max()

            #x = epi_img_data[:, :, int((n_k-1)/2)]

            x = np.expand_dims(x, axis=3)

            X[objindex] = x

            

            if objindex % 50 == 0:
                print ("loaded " + str(objindex) + "subjects")

                print (subject + " " + str(y))
        
        
    print (X.shape)
    print (Y.shape)
    
    print (countHC)
    print (countMCI)
    print (countAD)
        
    return X, Y

In [5]:
def load_dataset(dimension = '3d'):
    
    def load_mri_images(filename):
        global c
        
        data = np.load(filename)
        
        tmp = c
        c = tmp + 1
        
        print ('Loaded image set %d of 32.' %c)
        
        return data
    
    def imgwise_2d_scaling(data):
        #loop over patients
        for i in range(len(data)):
            for j in range(len(data[i][0])):
                max_val_2d = np.amax(data[i][0][j])

                data[i][0][j] = data[i][0][j].astype('float32')
                data[i][0][j] /= max_val_2d
                
        print ('Executed imagewise 2d scaling.')

        return data

    def imgwise_3d_scaling(data):
         #loop over patients
        for i in range(len(data)):
            max_val_3d = np.amax(data[i][0])

            data[i][0] = data[i][0].astype('float32')
            data[i][0] /= max_val_3d

        print ('Executed imagewise 3d scaling.')
        
        return data
    
    def reshape_mri_images(data):
        #Reshape the loaded dataset to the appropriate format.
        data = np.expand_dims(data,axis=1)
        
        if(dimension == '3d'):
            
            data = np.reshape(data, (-1, img_size_z, img_size_x, img_size_y, 1))
        
        print ('Reshaped images.')
        
        return data
    
    def load_mri_labels(filename, train_valid_test):
        data = pd.read_csv(filename)

        data = data.loc[data['train_valid_test'] == train_valid_test]
        
        data = np.asarray(data.diagnosis)
        data_new = np.array([])
        
        if(dimension == '2d'):
            for i, item in enumerate(data):
                data_temp = np.array([])
                for i in range(img_size_z):
                    data_temp = np.append(data_temp, item)  
                
                data_new = np.append(data_new, data_temp)
        else:
            data_new = data
            
        data_new = data_new.reshape((-1, 1))
        data_new = data_new.astype(np.int64)
        
        #labels start at 1, normalise them to start at 0.
        data_new = np.subtract(data_new, 1)
        
        data_new = np_utils.to_categorical(data_new, nb_classes)
        
        print ('Loaded labels.')

        return data_new
    
    train_data = load_mri_images('Data/img_array_train_6k_1.npy')
    for i in range(2,23):
        train_cur = load_mri_images('Data/img_array_train_6k_%d.npy' %i)
        train_data = np.vstack((train_data, train_cur))
    train_data = reshape_mri_images(train_data)
    
    val_data = load_mri_images('Data/img_array_valid_6k_1.npy')
    for i in range(2,6):
        valid_cur = load_mri_images('Data/img_array_valid_6k_%d.npy' %i)
        val_data = np.vstack((val_data, valid_cur))
    val_data = reshape_mri_images(val_data)
    
    test_data = load_mri_images('Data/img_array_test_6k_1.npy')
    for i in range(2,6):
        test_cur = load_mri_images('Data/img_array_test_6k_%d.npy' %i)
        test_data = np.vstack((test_data, test_cur))
    test_data = reshape_mri_images(test_data)
    
    if(dimension == '3d'):
        train_data = imgwise_3d_scaling(train_data)
        val_data = imgwise_3d_scaling(val_data)
        test_data = imgwise_3d_scaling(test_data)
    else:
        train_data = imgwise_2d_scaling(train_data)
        val_data = imgwise_2d_scaling(val_data)
        test_data = imgwise_2d_scaling(test_data)
        
    train_labels = load_mri_labels('adni_demographic_master_kaggle.csv', 0)
    val_labels = load_mri_labels('adni_demographic_master_kaggle.csv', 1)
    test_labels = load_mri_labels('adni_demographic_master_kaggle.csv', 2)
    
    # remove MCI data
    print(train_labels.shape)
    train_labels = np.delete(train_labels,1,axis=1)
    print(train_labels.shape)
    test_labels =  np.delete(test_labels,1,axis=1)
    val_labels =  np.delete(val_labels,1,axis=1)
    
    train_labels = np.where(train_labels[:,:]==[0,0],[1,0],train_labels[:,:])
    test_labels = np.where(test_labels[:,:]==[0,0],[1,0],test_labels[:,:])
    val_labels = np.where(val_labels[:,:]==[0,0],[1,0],val_labels[:,:])
    
    print ('Done.')
    
    return train_data, train_labels, test_data, test_labels, val_data, val_labels

In [6]:
train_data, train_labels, test_data, test_labels, val_data, val_labels = load_dataset(dimension = '3d')

Loaded image set 1 of 32.
Loaded image set 2 of 32.
Loaded image set 3 of 32.
Loaded image set 4 of 32.
Loaded image set 5 of 32.
Loaded image set 6 of 32.
Loaded image set 7 of 32.
Loaded image set 8 of 32.
Loaded image set 9 of 32.
Loaded image set 10 of 32.
Loaded image set 11 of 32.
Loaded image set 12 of 32.
Loaded image set 13 of 32.
Loaded image set 14 of 32.
Loaded image set 15 of 32.
Loaded image set 16 of 32.
Loaded image set 17 of 32.
Loaded image set 18 of 32.
Loaded image set 19 of 32.
Loaded image set 20 of 32.
Loaded image set 21 of 32.
Loaded image set 22 of 32.
Reshaped images.
Loaded image set 23 of 32.
Loaded image set 24 of 32.
Loaded image set 25 of 32.
Loaded image set 26 of 32.
Loaded image set 27 of 32.
Reshaped images.
Loaded image set 28 of 32.
Loaded image set 29 of 32.
Loaded image set 30 of 32.
Loaded image set 31 of 32.
Loaded image set 32 of 32.
Reshaped images.
Executed imagewise 3d scaling.
Executed imagewise 3d scaling.
Executed imagewise 3d scaling.
L

In [7]:
print(train_labels[0:10])
print(train_data.shape)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
(2109, 62, 96, 96, 1)


In [ ]:
#Dont run

X, Y = load_dataset(directory = "/home/ubuntu/project/Data/CombinedClean/")

In [ ]:
print (X.shape)

In [25]:
#Dont run

train_data = train_data[0:10]
train_labels = train_labels[0:10]
#test_data = X[205:245]
#test_labels = Y[205:245]
#val_data = X[205:306] #change
#val_labels = Y[205:306] #change

In [9]:
print (train_data.shape)

(2109, 62, 96, 96, 1)


In [8]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [24]:
#Builds keras 3D CNN model
def build_cnn(dimension = '3d', activation = 'softmax', heatmap = False, w_path = None, compile_model = True):
    input_3d = (img_size_z, img_size_x, img_size_y, 1)
    input_2d = (1, img_size_x, img_size_y)
    
    pool_3d = (2, 2, 2)
    pool_2d = (2, 2)
    
    def global_average_pooling(x):
        return K.mean(x, axis = (2, 3))

    def global_average_pooling_shape(input_shape):
        return input_shape[0:2]
    
    def build_conv_3d():
        model = Sequential()
        
        model.add(Conv3D(8, (3, 3, 3), name='conv1', input_shape=input_3d))
        model.add(MaxPooling3D(pool_size=pool_3d, name='pool1'))
        
        model.add(Conv3D(8, (3, 3, 3), name='conv2', input_shape=input_3d))
        model.add(MaxPooling3D(pool_size=pool_3d, name='pool2'))

        model.add(Conv3D(8, (3, 3, 3), name='conv3'))
        model.add(MaxPooling3D(pool_size=pool_3d, name='pool3'))
        
        #model.add(Convolution3D(32, 3, 3, 3, name='conv2b'))
        #model.add(MaxPooling3D(pool_size=pool_3d, name='pool2b'))

        #model.add(Convolution3D(32, 3, 3, 3, name='conv3a'))
        #model.add(MaxPooling3D(pool_size=pool_3d, name='pool3a'))
        
        #model.add(Convolution3D(16, 3, 3, 3, name='conv4'))
        #model.add(MaxPooling3D(pool_size=pool_3d, name='pool4'))
        
        return model
        
    def build_conv_2d():
        model = Sequential()
        
        model.add(Convolution2D(8, 3, 3, name='conv1', input_shape=input_2d))
        model.add(MaxPooling2D(pool_size=pool_2d, name='pool1'))

        model.add(Convolution2D(8, 3, 3, name='conv2'))
        model.add(MaxPooling2D(pool_size=pool_2d, name='pool2'))

        model.add(Convolution2D(8, 3, 3, name='conv3'))
        model.add(MaxPooling2D(pool_size=pool_2d, name='pool3'))
        
        return model
    
    if(dimension == '3d'):
        model = build_conv_3d()
    else:
        model = build_conv_2d()
        
    model.add(Flatten())
    model.add(Dense(2000, activation='relu', name='dense1'))
    model.add(Dropout(0.5, name='dropout1'))
    
    #model.add(Dense(2000, activation='relu', name='dense1a'))
    #model.add(Dropout(0.5, name='dropout1a'))

    model.add(Dense(500, activation='relu', name='dense2'))
    model.add(Dropout(0.5, name='dropout2'))

    model.add(Dense(2, activation='softmax', name='softmax'))
    
    '''model = Sequential()
    # 1st Volumetric Convolutional block
    model.add(Conv3D(8, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01), input_shape=input_3d))
    model.add(Conv3D(8, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    # 2nd Volumetric Convolutional block
    model.add(Conv3D(16, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
    model.add(Conv3D(16, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    # 3rd Volumetric Convolutional block
    model.add(Conv3D(32, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
    model.add(Conv3D(32, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
    model.add(Conv3D(32, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    # 4th Volumetric Convolutional block
    model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
    model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
    model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    # 1th Deconvolutional layer with batchnorm and dropout for regularization
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.7))
    # 2th Deconvolutional layer
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    # Output with softmax nonlinearity for classification
    model.add(Dense(1, activation='sigmoid'))
'''
    if w_path:
        model.load_weights(w_path)
        
    #opt = keras.optimizers.Adam(lr=1e-9, beta_1=0.9, beta_2=0.99, amsgrad=False)

    #opt = keras.optimizers.Adadelta(lr=1e-5)#clipnorm=1.)
    
    opt = keras.optimizers.Adam(lr=1e-6,beta_1=0.9, beta_2=0.999, amsgrad=False)
    
    #opt = SGD(lr=0.0000001)
    
    if(compile_model):
        model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
    
    print ('Done building model.')

    return model


In [25]:
model = build_cnn(dimension = '3d', compile_model = True)


Done building model.


In [26]:
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 60, 94, 94, 8)     224       
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 47, 47, 8)     0         
_________________________________________________________________
conv2 (Conv3D)               (None, 28, 45, 45, 8)     1736      
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 14, 22, 22, 8)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 12, 20, 20, 8)     1736      
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 6, 10, 10, 8)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4800)              0         
__________

In [27]:
# Tracks model learning rate
class SGDLearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        lr = K.eval(optimizer.lr * (1. / (1. + optimizer.decay * optimizer.iterations)))
        print (str('\nLR: {:.6f}\n').format(float(lr)))

In [28]:
# Fitting model architecture to data, also runs loss/accuracy tracker
def fit_model(model, v, train_data, train_labels, val_data, val_labels):
    model_weights_file = 'img_classifier_weights_%s.h5' %v
    epoch_weights_file = 'img_classifier_weights_%s_{epoch:02d}_{val_acc:.2f}.hdf5' %v
    model_file = 'img_classifier_model_%s.h5' %v
    history_file = 'img_classifier_history_%s.json' %v
    
    def save_model_and_weights():
        model.save(model_file)
        model.save_weights(model_weights_file)
        
        return 'Saved model and weights to disk!'

    def save_model_history(m):
        with open(history_file, 'w', encoding="utf8") as history_json_file:
            json.dump(m.history, history_json_file)
        
        return 'Saved model history to disk!'
    
    def visualise_accuracy(m):
        plt.plot(m.history['acc'])
        plt.plot(m.history['val_acc'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()
      
    def visualise_loss(m):
        plt.plot(m.history['loss'])
        plt.plot(m.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()
    
    def model_callbacks():
        checkpoint = ModelCheckpoint(epoch_weights_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=early_stopping_patience, verbose=1, mode='auto')
        lr_tracker = SGDLearningRateTracker()
        
        #roc = roc_callback(training_data=(train_data, train_labels),validation_data=(val_data,val_labels))
        
        return [checkpoint]
        
    callbacks_list = model_callbacks()
    
    y_ints = [y.argmax() for y in train_labels]
    
    #class_weights = class_weight.compute_class_weight('balanced',
                                                 #np.unique(y_ints),
                                                 #y_ints)
    #print (class_weights)
    
    m = model.fit(train_data,train_labels,batch_size=batch_size, epochs=nb_epoch, verbose=1,shuffle=True,validation_data=(val_data,val_labels),callbacks=callbacks_list)
    
    print (save_model_and_weights())
    print (save_model_history(m))
    
    visualise_accuracy(m)
    visualise_loss(m)
    
    return m

In [29]:
# Evaluates test data performance and creates confusion matrix
def evaluate_model(m, weights, test_data, test_labels):    
    def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, cm[i, j],
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
     
    plt.close('all')

    m.load_weights(weights)
    m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    print ("Done compiling model.")
    
    prediction = m.predict(test_data)
    prediction_labels = np_utils.to_categorical(np.argmax(prediction, axis=1), nb_classes)
    
    #print (prediction)
    
    pred = []
    for p in prediction:
        if float(p[0]) > 0.5:
            pred.append(1)
        else:
            pred.append(0)
    print (pred)
    
    print ('Accuracy on test data:', accuracy_score(test_labels, prediction_labels))

    print ('Classification Report')
    print (classification_report(test_labels, prediction_labels, target_names = ['CN', 'AD']))

    # Compute confusion matrix
    cnf_matrix = confusion_matrix(np.argmax(test_labels, axis=1), np.argmax(prediction, axis=1))
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes = class_names, normalize=False, title='Confusion matrix')

    plt.show()


In [30]:
#This will take time
m = fit_model(model, "v1", train_data, train_labels, val_data, val_labels)

Train on 2109 samples, validate on 435 samples
Epoch 1/50
2109/2109 [==============================] - 49s 23ms/step - loss: 6.2851 - acc: 0.8857 - val_loss: 5.1133 - val_acc: 1.0000

Epoch 00001: val_acc improved from -inf to 1.00000, saving model to img_classifier_weights_v1_01_1.00.hdf5
Epoch 2/50
2109/2109 [==============================] - 48s 23ms/step - loss: 5.5401 - acc: 0.9559 - val_loss: 5.1133 - val_acc: 1.0000

Epoch 00002: val_acc did not improve from 1.00000
Epoch 3/50
2109/2109 [==============================] - 47s 22ms/step - loss: 5.4112 - acc: 0.9682 - val_loss: 5.1133 - val_acc: 1.0000

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/50
1584/2109 [=====================>........] - ETA: 10s - loss: 5.1716 - acc: 0.9811

KeyboardInterrupt: 

In [ ]:
print (test_labels)

In [ ]:
#Load the fit model and evaluate performance on test
evaluate_model(model, 'img_classifier_weights_v1.h5', test_data, test_labels)

In [ ]:
print (test_labels)